# WebSocket L2 Client Demo
Connects to the mock WS and reconstructs L2 via diffs.

In [ ]:
import asyncio, json, websockets, numpy as np, matplotlib.pyplot as plt

## 1. Connect & receive snapshots

In [ ]:
async def run():
    uri='ws://localhost:8000/ws/orderbook/binance/BTCUSDT?depth=10&interval_ms=500'
    xs=[]
    async with websockets.connect(uri) as ws:
        for _ in range(20):
            msg=json.loads(await ws.recv())
            mid=(msg['bids'][0][0]+msg['asks'][0][0])/2
            xs.append(mid)
    return xs


## 2. Launch loop (works when mock_apis is running)

In [ ]:
try:
    arr=asyncio.get_event_loop().run_until_complete(run())
    plt.figure(); plt.plot(arr); plt.title('Mid via WS'); plt.show()
except Exception as e:
    print('Run mock_apis first:', e)


## 3-10. Reconstruction helpers

In [ ]:
def apply_deltas(levels, deltas):
    book=dict(levels)
    for p,s in deltas:
        if s<=0: book.pop(p,None)
        else: book[p]=s
    out=sorted(book.items(), key=lambda x: (-x[0]))
    return out
